# HP1 Plus and No HP1 models

Goal: Train elastic net regression models on 'HP1 Plus' and 'No HP1' datasets. Compare performance/interpretations to permutation datasets.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn
import warnings

## Load data

Read in the 'HP1 plus' and 'No HP1' model datasets.

In [2]:
# Dataset includes model data
plus_model = pd.read_csv('../datasets/model_inputs/hp1_plus_input.csv')
no_model = pd.read_csv('../datasets/model_inputs/no_hp1_input.csv')

plus_model.head()

,FBgn,log_TPM,HP1a,HP1B,HP1C,Mean_Accesibility,state.name_1,state.name_2,state.name_3,state.name_4,...,HP1C.GAGA_Sites,HP1C.GAGA_Avg_Score,HP1C.DRE_Sites,HP1C.DRE_Avg_Score,HP1C.Disco_Sites,HP1C.Disco_Avg_Score,A_B,A_C,B_C,A_B_C
0,FBgn0062565,0.197161,0.596583,-0.769971,-0.769417,2.04800,0,0,0,0,...,-1.538834,-4.313398,-0.769417,-6.516731,0.000000,0.000000,-0.459351,-0.459021,0.592429,0.353433
1,FBgn0053217,1.255029,2.769302,0.931543,1.072085,3.53125,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.579725,2.968927,0.998694,2.765684
2,FBgn0040372,0.622380,0.442630,1.026500,0.387516,2.59850,0,0,0,0,...,1.162549,1.466298,0.775032,3.534617,0.387516,5.225230,0.454360,0.171526,0.397786,0.176072
3,FBgn0000316,1.879004,0.576955,0.305769,0.251194,0.76900,0,0,0,0,...,0.753582,1.261552,0.502388,3.825009,0.251194,3.095365,0.176415,0.144928,0.076807,0.044314
4,FBgn0024989,-4.075113,0.611478,1.393595,1.637319,2.12500,0,0,0,1,...,3.274638,8.752215,0.000000,0.000000,3.274638,17.020193,0.852153,1.001185,2.281760,1.395246


## Train Test Split

As before, split the datasets before doing any other work. Keep a copy of data that includes FBgn identifiers which can be used for downstream analyses, but need to be excluded from model training.

In [3]:
from sklearn.model_selection import train_test_split

plus_copy = plus_model.copy()
no_copy = no_model.copy()
# Remove these variables until they are scaled properly
#genes_copy = genes_copy.drop(['Length', 'PInd'], axis = 1)

for set_ in (plus_copy, no_copy):
    set_.drop('log_TPM', axis=1, inplace=True)

plus_train, plus_test, yplus_train, yplus_test = train_test_split(plus_copy, plus_model['log_TPM'],
                                                                test_size = 0.2,
                                                                 random_state = 42)
plus_train.head()

no_train, no_test, yno_train, yno_test = train_test_split(no_copy, no_model['log_TPM'],
                                                         test_size=0.2, random_state=42)

plus_train2 = plus_train.drop(['FBgn'], axis = 1)
plus_test2 = plus_test.drop(['FBgn'], axis = 1)
no_train2 = no_train.drop(['FBgn'], axis = 1)
no_test2 = no_test.drop(['FBgn'], axis = 1)


In [4]:
# Test to retrieve chromatin state column indices
# Need this for column transformer
cstate_indices = []
for i in range(9):
    j = str(i+1)
    var_name = 'state.name_'+j
    k = plus_train2.columns.get_loc(var_name)
    cstate_indices.append(k)

other_indices = []
for i in range(len(plus_train2.columns)):
    if(i in cstate_indices):
        continue
    else:
        other_indices.append(i)

## Data Transformations and Grid Search

All the numerical variables need to be scaled and zero-centered. Chromatin state is already one-hot encoded and doesn't need this. After selectively scaling the numerical variables, the next step is to perform grid search to fine-tune hyperparameters.

In [5]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
from sklearn.compose import ColumnTransformer

parameters = {'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
             'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}

steps = list()
steps.append(['transformer', ColumnTransformer(transformers = [('num', StandardScaler(),
                                                              other_indices)],
                                              remainder='passthrough')])
steps.append(['model', ElasticNet()])
pipeline = Pipeline(steps = steps)

if __name__ == "__main__":
    plus_grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    plus_grid_search.fit(plus_train2, yplus_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % plus_grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = plus_grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Create a new pipeline for the no hp1 model
# Need to identify indices for column transformer
no_cstate_indices = []
for i in range(9):
    j = str(i+1)
    var_name = 'state.name_'+j
    k = no_train2.columns.get_loc(var_name)
    no_cstate_indices.append(k)

no_other_indices = []
for i in range(len(no_train2.columns)):
    if(i in cstate_indices):
        continue
    else:
        no_other_indices.append(i)

nosteps = list()
nosteps.append(['transformer', ColumnTransformer(transformers = [('num', StandardScaler(),
                                                                 no_other_indices)],
                                                remainder='passthrough')])
nosteps.append(['model', ElasticNet()])
no_pipeline = Pipeline(steps = nosteps)

if __name__ == "__main__":
    no_grid_search = GridSearchCV(no_pipeline, parameters, n_jobs=1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    no_grid_search.fit(no_train2, yno_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % no_grid_search.best_score_)
    print("Best parameters set:")
    best_no_parameters = no_grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_no_parameters[param_name]))

Performing grid search...
pipeline: ['transformer', 'model']
parameters:
{'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}
Fitting 3 folds for each of 25 candidates, totalling 75 fits


/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13.576165314655555, tolerance: 1.7380245312699107
  positive)
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:   12.1s finished


done in 13.223s

Best score: 0.566
Best parameters set:
	model__alpha: 0.1
	model__l1_ratio: 0.1
Performing grid search...
pipeline: ['transformer', 'model']
parameters:
{'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}
Fitting 3 folds for each of 25 candidates, totalling 75 fits


/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


done in 1.432s

Best score: 0.518
Best parameters set:
	model__alpha: 0.1
	model__l1_ratio: 0.1


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    1.4s finished


## Generate Predictions

Now that the model parameters are tuned, generate predictions for both the training and test set.

In [6]:
from sklearn.model_selection import cross_val_predict

plus_ytrain_pred = cross_val_predict(plus_grid_search.best_estimator_, plus_train2, 
                                     yplus_train, cv=10)

plus_ytest_pred = plus_grid_search.best_estimator_.predict(plus_test2)

no_ytrain_pred = cross_val_predict(no_grid_search.best_estimator_, no_train2,
                                 yno_train, cv=10)
no_ytest_pred = no_grid_search.best_estimator_.predict(no_test2)

/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.798644706419509, tolerance: 2.3869830377915533
  positive)
/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.78899508637187, tolerance: 2.4142506084724134
  positive)
/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.712968546000411, tolerance: 2.443193948958114
  positive)
/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converg

## Extract Coefficients

From the best fit model, extract coefficients.

In [7]:
plus_coefs = pd.DataFrame({'feature_names' : plus_train2.columns,
'feature_coefs' : plus_grid_search.best_estimator_.named_steps['model'].coef_})
plus_coefs.head()

no_coefs = pd.DataFrame({'feature_names': no_train2.columns,
                        'feature_coefs': no_grid_search.best_estimator_.named_steps['model'].coef_})
print(no_train2.columns)

Index(['Mean_Accesibility', 'state.name_1', 'state.name_2', 'state.name_3',
       'state.name_4', 'state.name_5', 'state.name_6', 'state.name_7',
       'state.name_8', 'state.name_9',
       ...
       'TTG', 'TTT', 'exon_num', 'density', 'GAGA_Sites', 'GAGA_Avg_Score',
       'DRE_Sites', 'DRE_Avg_Score', 'Disco_Sites', 'Disco_Avg_Score'],
      dtype='object', length=105)


## Write Output

Now write out the prediction and coefficient datasets

In [8]:
plus_coefs.to_csv('../datasets/model_outputs/HP1_plus_coefficients_no_RFE.csv',
                 index = False)
plus_train_preds = pd.DataFrame({'FBgn': plus_train['FBgn'],
                                'Y_true': yplus_train,
                                'Y_pred': plus_ytrain_pred})
plus_train_preds.to_csv('../datasets/model_outputs/HP1_plus_train_predictions_no_RFE.csv',
                       index = False)

no_coefs.to_csv('../datasets/model_outputs/No_HP1_coefficients_no_RFE.csv',
               index = False)
no_train_preds = pd.DataFrame({'FBgn': no_train['FBgn'],
                              'Y_true': yno_train,
                              'Y_pred': no_ytrain_pred})
no_train_preds.to_csv('../datasets/model_outputs/No_HP1_train_predictions_no_RFE.csv',
                     index=False)

plus_test_preds = pd.DataFrame({'FBgn': plus_test['FBgn'],
                               'Y_true': yplus_test,
                               'Y_pred': plus_ytest_pred})
plus_test_preds.to_csv('../datasets/model_outputs/HP1_plus_test_predictions_no_RFE.csv',
                      index=False)

no_test_preds = pd.DataFrame({'FBgn': no_test['FBgn'],
                             'Y_true': yno_test,
                             'Y_pred': no_ytest_pred})
no_test_preds.to_csv('../datasets/model_outputs/No_HP1_test_predictions_no_RFE.csv',
                    index=False)

## Permutations

Now, last step is to run through model fitting, generate predictions and extract coefficients with the permuted datasets.

In [9]:
# load data
plus_perm = pd.read_csv('../datasets/model_inputs/HP1_Plus_Permuted_Input.csv')
no_perm = pd.read_csv('../datasets/model_inputs/No_HP1_Permuted_Inputs.csv')

# train test split
plus_copy = plus_model.copy()
no_copy = no_model.copy()

pperm_copy = plus_perm.copy()
nperm_copy = no_perm.copy()

for set_ in (pperm_copy, nperm_copy):
    set_.drop('log_TPM', axis=1, inplace=True)

pperm_train, pperm_test, ypp_train, ypp_test = train_test_split(pperm_copy, plus_perm['log_TPM'],
                                                               test_size = 0.2, random_state=42)
nperm_train, nperm_test, ynp_train, ynp_test = train_test_split(nperm_copy, no_perm['log_TPM'],
                                                               test_size = 0.2, random_state=42)
# pipeline, parameter tuning, model fit
steps = list()
steps.append(('scaler', StandardScaler()))
steps.append(('model', ElasticNet()))
perm_pipeline = Pipeline(steps = steps)

if __name__ == "__main__":
    pperm_grid_search = GridSearchCV(perm_pipeline, parameters, n_jobs=1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    pperm_grid_search.fit(pperm_train, ypp_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % pperm_grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = pperm_grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

if __name__ == "__main__":
    nperm_grid_search = GridSearchCV(perm_pipeline, parameters, n_jobs=1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    nperm_grid_search.fit(nperm_train, ynp_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % nperm_grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = nperm_grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

# generate predictions
ypp_train_pred = cross_val_predict(pperm_grid_search.best_estimator_, pperm_train,
                                  ypp_train, cv=10)
ypp_test_pred = pperm_grid_search.best_estimator_.predict(pperm_test)

ynp_train_pred = cross_val_predict(nperm_grid_search.best_estimator_, nperm_train,
                                  ynp_train, cv=10)
ynp_test_pred = nperm_grid_search.best_estimator_.predict(nperm_test)

# extract coefficients
pperm_coefs = pd.DataFrame({'Feature_Name': pperm_train.columns,
                           'Feature_Coef': pperm_grid_search.best_estimator_.named_steps['model'].coef_})

nperm_coefs = pd.DataFrame({'Feature_Name': nperm_train.columns,
                           'Feature_Coef': nperm_grid_search.best_estimator_.named_steps['model'].coef_})

# write output
pperm_preds_train = pd.DataFrame({'Y_true': ypp_train, 'Y_pred': ypp_train_pred})
pperm_preds_test = pd.DataFrame({'Y_true': ypp_test, 'Y_pred': ypp_test_pred})
nperm_preds_train = pd.DataFrame({'Y_true': ynp_train, 'Y_pred': ynp_train_pred})
nperm_preds_test = pd.DataFrame({'Y_true': ynp_test, 'Y_pred': ynp_test_pred})

pperm_coefs.to_csv('../datasets/model_outputs/HP1_Plus_Permutations_Coefs.csv', index=False)
nperm_coefs.to_csv('../datasets/model_outputs/No_HP1_Permutations_Coefs.csv', index=False)

pperm_preds_train.to_csv('../datasets/model_outputs/HP1_Plus_Permutation_TrainPreds.csv',
                        index=False)
pperm_preds_test.to_csv('../datasets/model_outputs/HP1_Plus_Permutation_TestPreds.csv',
                       index=False)

nperm_preds_train.to_csv('../datasets/model_outputs/No_HP1_Permutation_TrainPreds.csv',
                        index=False)
nperm_preds_test.to_csv('../datasets/model_outputs/No_HP1_Permutation_TestPreds.csv',
                       index=False)

Performing grid search...
pipeline: ['transformer', 'model']
parameters:
{'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}
Fitting 3 folds for each of 25 candidates, totalling 75 fits


/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    4.7s finished
/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


done in 4.791s

Best score: 0.081
Best parameters set:
	model__alpha: 0.9
	model__l1_ratio: 0.1
Performing grid search...
pipeline: ['transformer', 'model']
parameters:
{'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}
Fitting 3 folds for each of 25 candidates, totalling 75 fits


[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    0.7s finished


done in 0.731s

Best score: 0.000
Best parameters set:
	model__alpha: 0.1
	model__l1_ratio: 0.9


## Main effects only

Train a model without interaction effects and examine coefficient values and accuracy.

In [23]:
mains = pd.read_csv('../datasets/model_inputs/main_effects_only_plus_inputs.csv')
mains_copy = mains.copy()
mains_copy.head()
# Remove these variables until they are scaled properly
#genes_copy = genes_copy.drop(['Length', 'PInd'], axis = 1)
mains_copy.drop('log_TPM', axis=1, inplace=True)

mains_train, mains_test, ymain_train, ymain_test = train_test_split(mains_copy, mains['log_TPM'],
                                                                test_size = 0.2,
                                                                 random_state = 42)


mains_train2 = mains_train.drop(['FBgn'], axis = 1)

cstate_indices = []
for i in range(9):
    j = str(i+1)
    var_name = 'state.name_'+j
    k = mains_train2.columns.get_loc(var_name)
    cstate_indices.append(k)

other_indices = []
for i in range(len(mains_train2.columns)):
    if(i in cstate_indices):
        continue
    else:
        other_indices.append(i)
        
parameters = {'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
             'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}

steps = list()
steps.append(['transformer', ColumnTransformer(transformers = [('num', StandardScaler(),
                                                              other_indices)],
                                              remainder='passthrough')])
steps.append(['model', ElasticNet()])
pipeline = Pipeline(steps = steps)

if __name__ == "__main__":
    mains_grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    mains_grid_search.fit(mains_train2, ymain_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % mains_grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = mains_grid_search.best_estimator_.get_params()
    
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

mains_ytrain_pred = cross_val_predict(mains_grid_search.best_estimator_, mains_train2, 
                                     ymain_train, cv=10)

mains_coefs = pd.DataFrame({'feature_names' : mains_train2.columns,
'feature_coefs' : mains_grid_search.best_estimator_.named_steps['model'].coef_})
mains_coefs.head()


mains_coefs.to_csv('../datasets/model_outputs/HP1_plus_coefficients_main_effects_only.csv',
                 index = False)
mains_train_preds = pd.DataFrame({'FBgn': mains_train['FBgn'],
                                'Y_true': ymain_train,
                                'Y_pred': mains_ytrain_pred})
plus_train_preds.to_csv('../datasets/model_outputs/HP1_plus_train_predictions_main_effects_only.csv',
                       index = False)

Performing grid search...
pipeline: ['transformer', 'model']
parameters:
{'model__alpha': [0.1, 0.3, 0.5, 0.7, 0.9],
 'model__l1_ratio': [0.1, 0.25, 0.5, 0.75, 0.9]}
Fitting 3 folds for each of 25 candidates, totalling 75 fits


/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:    1.3s finished


done in 1.318s

Best score: 0.556
Best parameters set:
	model__alpha: 0.1
	model__l1_ratio: 0.1


In [24]:
## Simulated data
a_cpr11b = pd.read_csv('../datasets/dcas9_simulations/cpr11b_hp1a_hp1_plus_inputs.csv')
b_cpr11b = pd.read_csv('../datasets/dcas9_simulations/cpr11b_hp1b_hp1_plus_inputs.csv')
c_cpr11b = pd.read_csv('../datasets/dcas9_simulations/cpr11b_hp1c_hp1_plus_inputs.csv')

a_dgt3 = pd.read_csv('../datasets/dcas9_simulations/dgt3_hp1a_hp1_plus_inputs.csv')
b_dgt3 = pd.read_csv('../datasets/dcas9_simulations/dgt3_hp1b_hp1_plus_inputs.csv')
c_dgt3 = pd.read_csv('../datasets/dcas9_simulations/dgt3_hp1c_hp1_plus_inputs.csv')

a_ceca1 = pd.read_csv('../datasets/dcas9_simulations/ceca1_hp1a_hp1_plus_inputs.csv')
b_ceca1 = pd.read_csv('../datasets/dcas9_simulations/ceca1_hp1b_hp1_plus_inputs.csv')
c_ceca1 = pd.read_csv('../datasets/dcas9_simulations/ceca1_hp1c_hp1_plus_inputs.csv')

a_mtk = pd.read_csv('../datasets/dcas9_simulations/mtk_hp1a_hp1_plus_inputs.csv')
b_mtk = pd.read_csv('../datasets/dcas9_simulations/mtk_hp1b_hp1_plus_inputs.csv')
c_mtk = pd.read_csv('../datasets/dcas9_simulations/mtk_hp1c_hp1_plus_inputs.csv')

a_egr = pd.read_csv('../datasets/dcas9_simulations/egr_hp1a_hp1_plus_inputs.csv')
a_pyr = pd.read_csv('../datasets/dcas9_simulations/pyr_hp1a_hp1_plus_inputs.csv')
a_mats = pd.read_csv('../datasets/dcas9_simulations/mats_hp1a_hp1_plus_inputs.csv')

b_rab3 = pd.read_csv('../datasets/dcas9_simulations/rab3_hp1b_hp1_plus_inputs.csv')
b_shaw = pd.read_csv('../datasets/dcas9_simulations/shaw_hp1b_hp1_plus_inputs.csv')
b_cg76 = pd.read_csv('../datasets/dcas9_simulations/cg76_hp1b_hp1_plus_inputs.csv')

c_alpha = pd.read_csv('../datasets/dcas9_simulations/alpha_hp1c_hp1_plus_inputs.csv')
c_cg26 = pd.read_csv('../datasets/dcas9_simulations/cg26_hp1c_hp1_plus_inputs.csv')

In [26]:
a_cpr11b_preds = plus_grid_search.best_estimator_.predict(a_cpr11b)
b_cpr11b_preds = plus_grid_search.best_estimator_.predict(b_cpr11b)
c_cpr11b_preds = plus_grid_search.best_estimator_.predict(c_cpr11b)

a_dgt3_preds = plus_grid_search.best_estimator_.predict(a_dgt3)
b_dgt3_preds = plus_grid_search.best_estimator_.predict(b_dgt3)
c_dgt3_preds = plus_grid_search.best_estimator_.predict(c_dgt3)

a_ceca1_preds = plus_grid_search.best_estimator_.predict(a_ceca1)
b_ceca1_preds = plus_grid_search.best_estimator_.predict(b_ceca1)
c_ceca1_preds = plus_grid_search.best_estimator_.predict(c_ceca1)

a_mtk_preds = plus_grid_search.best_estimator_.predict(a_mtk)
b_mtk_preds = plus_grid_search.best_estimator_.predict(b_mtk)
c_mtk_preds = plus_grid_search.best_estimator_.predict(c_mtk)

a_egr_preds = plus_grid_search.best_estimator_.predict(a_egr)
a_pyr_preds = plus_grid_search.best_estimator_.predict(a_pyr)
a_mats_preds = plus_grid_search.best_estimator_.predict(a_mats)

b_rab3_preds = plus_grid_search.best_estimator_.predict(b_rab3)
b_shaw_preds = plus_grid_search.best_estimator_.predict(b_shaw)
b_cg76_preds = plus_grid_search.best_estimator_.predict(b_cg76)

c_alpha_preds = plus_grid_search.best_estimator_.predict(c_alpha)
c_cg26_preds = plus_grid_search.best_estimator_.predict(c_cg26)

cpr11b_out = pd.DataFrame({'HP1a': a_cpr11b_preds,
                          'HP1B': b_cpr11b_preds,
                          'HP1C': c_cpr11b_preds})
dgt3_out = pd.DataFrame({'HP1a': a_dgt3_preds,
                        'HP1B': b_dgt3_preds,
                        'HP1C': c_dgt3_preds})
ceca1_out = pd.DataFrame({'HP1a': a_ceca1_preds,
                         'HP1B': b_ceca1_preds,
                         'HP1C': c_ceca1_preds})
mtk_out = pd.DataFrame({'HP1a': a_mtk_preds,
                       'HP1B': b_mtk_preds,
                       'HP1C': c_mtk_preds})

hp1a_out = pd.DataFrame({'egr': a_egr_preds,
                       'pyr': a_pyr_preds,
                       'mats': a_mats_preds})
hp1b_out = pd.DataFrame({'rab3': b_rab3_preds,
                       'shaw': b_shaw_preds,
                       'cg76': b_cg76_preds})
hp1c_out = pd.DataFrame({'alpha': c_alpha_preds,
                       'cg26': c_cg26_preds})

cpr11b_out.to_csv('../datasets/model_outputs/cpr11b_hp1_plus_simulated.csv')
dgt3_out.to_csv('../datasets/model_outputs/dgt3_hp1_plus_simulated.csv')
ceca1_out.to_csv('../datasets/model_outputs/ceca1_hp1_plus_simulated.csv')
mtk_out.to_csv('../datasets/model_outputs/mtk_hp1_plus_simulated.csv')
hp1a_out.to_csv('../datasets/model_outputs/hp1a_hp1_plus_simulated.csv')
hp1b_out.to_csv('../datasets/model_outputs/hp1b_hp1_plus_simulated.csv')
hp1c_out.to_csv('../datasets/model_outputs/hp1c_hp1_plus_simulated.csv')

In [12]:
baselines = pd.read_csv('../datasets/model_inputs/HP1_Plus_dcas9_baselines.csv')
baseline_preds = plus_grid_search.best_estimator_.predict(baselines)
print(baseline_preds)

[-460.35890424 -145.66383712 -488.75583917 -309.61267959]


In [13]:
max(nperm_coefs['Feature_Coef'])


0.03137980730178763

In [14]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.feature_selection import RFE

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation:", scores.std())
    
rfe_selector = RFE(estimator=ElasticNet(alpha = 0.1, l1_ratio = 0.1),
                   n_features_to_select = 15, step = 1)
rfe_selector.fit(train_set2, y_train)

train_set2.columns[rfe_selector.get_support()]
    
#elastic_net = ElasticNet(alpha = 0.1, l1_ratio = .5)

#scores = cross_val_score(elastic_net, train_set2, y_train,
                        #scoring = "neg_mean_squared_error", cv=10)
#net_rmse_scores = np.sqrt(-scores)

#display_scores(net_rmse_scores)

NameError: name 'train_set2' is not defined

In [ ]:
rfe_train = train_set2[train_set2.columns[rfe_selector.get_support()]]
enet = ElasticNet(alpha = 0.1, l1_ratio = 0.5)
y_pred_plus_rfe = cross_val_predict(enet, rfe_train, y_train, cv=10)
y_pred_plus = cross_val_predict(enet, train_set2, y_train, cv=10)

Resid = y_pred_plus - y_train
resid_rfe = y_pred_plus_rfe - y_train

# Compare - did the RFE improve performance?
print(sum(Resid**2))
print(sum(resid_rfe**2))

In [7]:
data = {'FBgn': train_set['FBgn'],
       'HP1a': train_set['HP1a'],
       'HP1B': train_set['HP1B'],
        'HP1C': train_set['HP1C'],
       'B_C': train_set['B_C'],
        'state.name_1': train_set['state.name_1'],
        'state.name_2': train_set['state.name_2'],
        'state.name_4': train_set['state.name_4'],
        'state.name_9': train_set['state.name_9'],
        'CTC': train_set['CTC'],
        'GGA': train_set['GGA'],
        'GTC': train_set['GTC'],
        'DRE_Sites': train_set['DRE_Sites'],
        'DRE_Avg_Score': train_set['DRE_Avg_Score'],
        'Disco_Sites': train_set['Disco_Sites'],
        'Disco_Avg_Score': train_set['Disco_Avg_Score'],
       'Y_true': y_train,
       'Y_Pred_Full': y_pred_plus_rfe,}
       
        
output = pd.DataFrame(data)
output.head()

,FBgn,HP1a,HP1B,HP1C,B_C,state.name_1,state.name_2,state.name_4,state.name_9,CTC,GGA,GTC,DRE_Sites,DRE_Avg_Score,Disco_Sites,Disco_Avg_Score,Y_true,Y_Pred_Full
7828,FBgn0260960,1.976924,3.629032,4.837500,17.555440,0,0,0,0,1,1,0,0,0.00000,0,0.00000,1.422502,0.611836
2456,FBgn0052000,2.453688,0.973861,1.645251,1.602246,0,0,0,0,0,2,0,0,0.00000,0,0.00000,0.097486,0.519560
8981,FBgn0004228,-1.228404,-0.178896,-0.256333,0.045857,0,1,0,0,5,2,0,2,3.34091,0,0.00000,-1.276521,0.083403
2455,FBgn0005278,3.115202,2.214164,1.933721,4.281574,0,1,0,0,1,0,2,0,0.00000,0,0.00000,0.798414,0.869878
8810,FBgn0031053,0.784784,2.027456,1.988963,4.032536,0,0,0,0,0,1,3,2,11.26515,3,6.23118,0.670263,1.327450


In [8]:
output.to_csv('HP1_Plus_EN_Predictions.csv', index=False)

In [9]:
# Now do a genomics model without HP1
nohp1_data = pd.read_csv('no-hp1-model-data.csv')
nohp1_data.head()

,FBgn,log_S2,Density,GC.Promoter,GC.Body,Strand,PInd,state.name_1,state.name_2,state.name_3,...,TTC,TTG,TTT,Mean_Accesibility,DRE_Sites,DRE_Avg_Score,GAGA_Sites,GAGA_Avg_Score,Disco_Sites,Disco_Avg_Score
0,FBgn0062565,0.064436,1818,0.536888,0.51,2,2.257467,0,0,0,...,2,4,3,2.04800,1,8.46970,2,5.606060,0,0.00000
1,FBgn0053217,0.523959,1002,0.320990,0.35,2,40.465151,0,0,0,...,2,1,6,3.53125,0,0.00000,0,0.000000,0,0.00000
2,FBgn0040372,0.243543,1542,0.419050,0.47,2,7.004761,0,0,0,...,4,6,10,2.59850,2,9.12121,3,3.783837,1,13.48390
3,FBgn0000316,0.789471,1542,0.421467,0.34,2,5.660925,0,0,0,...,4,2,4,0.76900,2,15.22730,3,5.022220,1,12.32260
4,FBgn0024989,-1.916831,1542,0.366266,0.36,2,69.763614,0,0,0,...,1,5,3,2.12500,0,0.00000,2,5.345455,2,10.39516


In [10]:
nohp1_copy = nohp1_data.copy()
# Remove these variables until they are scaled properly
#nohp1_copy = nohp1_copy.drop(['Length', 'PInd'], axis = 1)

nohp1_train, nohp1_test = train_test_split(nohp1_copy, test_size = 0.2,
                                       random_state = 42)


no_y_train = nohp1_train['log_S2']
no_y_test = nohp1_test['log_S2']

for set_ in (nohp1_train, nohp1_test):
    set_.drop("log_S2", axis=1, inplace=True)

    # Just keeping a version of each of these that retains the FBgn
nohp1_train2 = nohp1_train.drop(['FBgn'], axis = 1)
nohp1_test2 = nohp1_test.drop(['FBgn'], axis = 1)

for (columnName, columnData) in nohp1_train2.iteritems():
    model_data[columnName] = model_data[columnName].astype('float32')
    
nohp1_train2.head()

/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Density,GC.Promoter,GC.Body,Strand,PInd,state.name_1,state.name_2,state.name_3,state.name_4,state.name_5,...,TTC,TTG,TTT,Mean_Accesibility,DRE_Sites,DRE_Avg_Score,GAGA_Sites,GAGA_Avg_Score,Disco_Sites,Disco_Avg_Score
7828,912,0.396569,0.41,1,10.452183,0,0,1,0,0,...,2,2,10,15.0175,0,0.00000,0,0.00000,0,0.00000
2456,1521,0.333945,0.31,2,2.493440,0,0,0,0,0,...,2,4,12,16.4100,0,0.00000,0,0.00000,0,0.00000
8981,896,0.359779,0.43,1,2.380867,0,1,0,0,0,...,4,2,7,3.5525,2,3.34091,1,5.74545,0,0.00000
2455,2,0.410679,0.27,2,0.052570,0,1,0,0,0,...,1,8,12,22.3575,0,0.00000,0,0.00000,0,0.00000
8810,1812,0.500994,0.44,1,1.061447,0,0,0,0,1,...,5,4,1,3.5755,2,11.26515,1,1.49697,3,6.23118


In [11]:

elastic_net_gsearch.fit(nohp1_train2, no_y_train)
print(elastic_net_gsearch.best_params_)

{'alpha': 0.1, 'l1_ratio': 0.1}


In [12]:
rfe_selector2 = RFE(estimator=ElasticNet(alpha = 0.1, l1_ratio = .1),
                   n_features_to_select = 15, step = 1)
rfe_selector2.fit(nohp1_train2, no_y_train)

nohp1_train2.columns[rfe_selector2.get_support()]

Index(['state.name_1', 'state.name_2', 'state.name_3', 'state.name_4',
       'state.name_9', 'CTC', 'GGA', 'GGC', 'GTC', 'TCC', 'Mean_Accesibility',
       'DRE_Sites', 'DRE_Avg_Score', 'Disco_Sites', 'Disco_Avg_Score'],
      dtype='object')

In [27]:
rfe_no_train = nohp1_train2[nohp1_train2.columns[rfe_selector2.get_support()]]

no_y_pred_rfe = cross_val_predict(enet, rfe_no_train, no_y_train, cv=10)

resid_rfe = no_y_pred_rfe - y_train

no_y_pred = cross_val_predict(enet, nohp1_train2, no_y_train, cv=10)
no_resid = no_y_pred - no_y_train

print(sum(resid_rfe**2))
print(sum(no_resid**2))

5120.639228182539
5099.058965551181


In [28]:
# Write out predictions for this dataset
data = {'FBgn': nohp1_train['FBgn'],
       'Y_true': no_y_train,
       'Y_Pred_Full': no_y_pred_rfe,}
       
        
output = pd.DataFrame(data)
output.head()

,FBgn,Y_true,Y_Pred_Full
7828,FBgn0260960,1.422502,0.475136
2456,FBgn0052000,0.097486,0.540815
8981,FBgn0004228,-1.276521,0.157085
2455,FBgn0005278,0.798414,0.740574
8810,FBgn0031053,0.670263,0.825742


In [29]:
output.to_csv('Genome_Baseline_Model_Predictions.csv')

In [30]:
# Are models robust to randomly permuted data?
mlplus_rand = pd.read_csv('HP1plus_randomized.csv')
hp1only_rand = pd.read_csv('HP1only_randomized.csv')
nohp1_rand = pd.read_csv('NoHP1_randomized.csv')

rplus_train, rplus_test = train_test_split(mlplus_rand, test_size = 0.2,
                                       random_state = 42)
rhp1_train, rhp1_test = train_test_split(hp1only_rand, test_size = 0.2,
                                       random_state = 42)
rno_train, rno_test = train_test_split(nohp1_rand, test_size = 0.2,
                                       random_state = 42)

rplus_y = rplus_train['log_S2']
rhp1_y = rhp1_train['log_S2']
rno_y = rno_train['log_S2']

for set_ in (rplus_train, rhp1_train, rno_train):
    set_.drop("log_S2", axis=1, inplace=True)
    
rhp1_train.head()

/Users/jack/miniconda3/envs/tf2/lib/python3.7/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,HP1a,HP1B,HP1C,A_B,A_C,B_C,A_B_C
7828,7.174321,1.976924,34.705778,9.563371,17.555440,4.837500,3.629032
2456,3.931411,4.036933,0.973861,1.602246,1.645251,2.453688,2.389551
8981,-0.056331,0.314880,-0.256333,0.219757,0.045857,-1.228404,-0.178896
2455,2.214164,6.897567,4.281574,3.115202,1.933721,6.023930,13.337967
8810,0.784784,1.988963,4.032536,2.027456,1.560908,3.164672,1.591116


In [31]:
# rplus grid search
elastic_net_gsearch.fit(rplus_train, rplus_y)
print(elastic_net_gsearch.best_params_)

{'alpha': 0.9, 'l1_ratio': 0.9}


In [32]:
rfe_selector = RFE(estimator=ElasticNet(alpha = 0.9, l1_ratio = .9),
                   n_features_to_select = 15, step = 1)
rfe_selector.fit(rplus_train, rplus_y)

rplus_train.columns[rfe_selector.get_support()]

Index(['HP1B', 'T', 'AC', 'CA', 'CT', 'TG', 'AAG', 'AAT', 'ACA', 'ACC', 'ATA',
       'GAT', 'GTA', 'TAA', 'TCT'],
      dtype='object')

In [33]:
# Randomized mlplus model performance
rfe_rplus = rplus_train[rplus_train.columns[rfe_selector.get_support()]]
rplus_net = ElasticNet(alpha = 0.9, l1_ratio = 0.9)

rplus_y_pred = cross_val_predict(rplus_net, rfe_rplus, rplus_y, cv=10)

rplus_resid = rplus_y_pred - rplus_y

print(sum(rplus_resid**2))

9054.864271710625


In [34]:
elastic_net_gsearch.fit(rno_train, rno_y)
print(elastic_net_gsearch.best_params_)

{'alpha': 0.9, 'l1_ratio': 0.9}


In [35]:
rfe_selector = RFE(estimator=ElasticNet(alpha = 0.9, l1_ratio = .9),
                   n_features_to_select = 15, step = 1)
rfe_selector.fit(rno_train, rno_y)

rno_train.columns[rfe_selector.get_support()]

Index(['Density', 'Strand', 'state.name_2', 'state.name_5', 'C', 'AG', 'CG',
       'AAG', 'ATT', 'CAA', 'CCA', 'TAC', 'TCG', 'TTT', 'Disco_Avg_Score'],
      dtype='object')

In [36]:
# Randomized no hp1 model performance
rfe_rno = rno_train[rno_train.columns[rfe_selector.get_support()]]

# Parameters are the same
rno_y_pred = cross_val_predict(rplus_net, rfe_rno, rno_y, cv=10)

rno_resid = rno_y_pred - rno_y

print(sum(rno_resid**2))

9089.705418069998


In [37]:
# Randomized HP1 only model performance
rhp1_y_pred = cross_val_predict(rplus_net, rhp1_train, rhp1_y, cv=10)

rhp1_resid = rhp1_y_pred - rhp1_y

print(sum(rhp1_resid**2))

9098.801037789628


In [38]:
# Now - write outputs for randomized datasets
# HP1 plus permutation output
rplus_out = {'Y_pred': rhp1_y_pred,
             'Y_true': rplus_y}
rplus_out_df = pd.DataFrame(rplus_out)
rplus_out_df.to_csv('HP1_Plus_Permutation_output.csv', index = False)

# No HP1 permutation output
rno_out = {'Y_pred': rno_y_pred,
          'Y_true': rno_y}
rno_out_df = pd.DataFrame(rno_out)
rno_out_df.to_csv('No_HP1_Permutation_output.csv', index = False)

In [45]:
# Test sets
# HP1 Plus
# Fit model on entire training set
hp1_plus_enet = ElasticNet(alpha = 0.1, l1_ratio = 0.1)


hp1_plus_enet.fit(train_set2[train_set2.columns[rfe_selector.get_support()]], y_train)
rfe_test_plus = test_set2[train_set2.columns[rfe_selector.get_support()]]

hp1_plus_y_test_preds = hp1_plus_enet.predict(rfe_test_plus)
hp1_plus_test_out = {'Y_true': y_test,
                    'Y_pred': hp1_plus_y_test_preds,
                    'FBgn': test_set['FBgn']}
pd.DataFrame(hp1_plus_test_out).to_csv('HP1_Plus_Model_Test_Predictions.csv', index = False)

no_hp1_enet = ElasticNet(alpha = 0.1, l1_ratio = 0.1)
no_hp1_enet.fit(nohp1_train2[nohp1_train2.columns[rfe_selector2.get_support()]], no_y_train)
rfe_test_no = nohp1_test2[nohp1_train2.columns[rfe_selector2.get_support()]]

no_hp1_test_preds = no_hp1_enet.predict(rfe_test_no)
no_test_out = {'Y_true': no_y_test,
               'Y_pred': no_hp1_test_preds,
               'FBgn': nohp1_test['FBgn']}
pd.DataFrame(no_test_out).to_csv('No_HP1_Model_Test_Predictions.csv', index = False)

In [46]:
# Coefficients
hp1_plus_coefs = pd.DataFrame({'Features': train_set2.columns[rfe_selector.get_support()],
                              'Coefficient': hp1_plus_enet.coef_})
hp1_plus_coefs.to_csv('HP1_Plus_Model_Coefficients.csv', index = False)

no_hp1_coefs = pd.DataFrame({'Features': nohp1_train2.columns[rfe_selector2.get_support()],
                           'Coefficient': no_hp1_enet.coef_})
no_hp1_coefs.to_csv('No_HP1_Model_Coefficients.csv', index = False)

In [52]:
# Simulations

dgt3_a = pd.read_csv('dgt3_HP1a_HP1Plus_sim.csv')
dgt3_b = pd.read_csv('dgt3_HP1B_HP1Plus_sim.csv')
dgt3_c = pd.read_csv('dgt3_HP1C_HP1Plus_sim.csv')

a_dgt3_preds = hp1_plus_enet.predict(dgt3_a)
b_dgt3_preds = hp1_plus_enet.predict(dgt3_b)
c_dgt3_preds = hp1_plus_enet.predict(dgt3_c)

dgt3_predictions = pd.DataFrame({'HP1a': a_dgt3_preds,
                                'HP1B': b_dgt3_preds,
                                'HP1C': c_dgt3_preds})
dgt3_predictions.to_csv('dgt3_HP1Plus_sim_predictions.csv', index = False)

In [53]:
cpr11b_a = pd.read_csv('Cpr11B_HP1a_HP1Plus_sim.csv')
cpr11b_b = pd.read_csv('Cpr11B_HP1B_HP1Plus_sim.csv')
cpr11b_c = pd.read_csv('Cpr11B_HP1C_HP1Plus_sim.csv')

a_Cpr11B_preds = hp1_plus_enet.predict(cpr11b_a)
b_Cpr11B_preds = hp1_plus_enet.predict(cpr11b_b)
c_Cpr11B_preds = hp1_plus_enet.predict(cpr11b_c)

Cpr11B_predictions = pd.DataFrame({'HP1a': a_Cpr11B_preds,
                                'HP1B': b_Cpr11B_preds,
                                'HP1C': c_Cpr11B_preds})
Cpr11B_predictions.to_csv('Cpr11B_HP1Plus_sim_predictions.csv', index = False)

In [54]:
ceca1_a = pd.read_csv('CecA1_HP1a_HP1Plus_sim.csv')
ceca1_b = pd.read_csv('CecA1_HP1B_HP1Plus_sim.csv')
ceca1_c = pd.read_csv('CecA1_HP1C_HP1Plus_sim.csv')

a_CecA1_preds = hp1_plus_enet.predict(ceca1_a)
b_CecA1_preds = hp1_plus_enet.predict(ceca1_b)
c_CecA1_preds = hp1_plus_enet.predict(ceca1_c)

CecA1_predictions = pd.DataFrame({'HP1a': a_CecA1_preds,
                                'HP1B': b_CecA1_preds,
                                'HP1C': c_CecA1_preds})
CecA1_predictions.to_csv('CecA1_HP1Plus_sim_predictions.csv', index = False)

In [55]:
mtk_a = pd.read_csv('Mtk_HP1a_HP1Plus_sim.csv')
mtk_b = pd.read_csv('Mtk_HP1B_HP1Plus_sim.csv')
mtk_c = pd.read_csv('Mtk_HP1C_HP1Plus_sim.csv')

a_Mtk_preds = hp1_plus_enet.predict(cpr11b_a)
b_Mtk_preds = hp1_plus_enet.predict(cpr11b_b)
c_Mtk_preds = hp1_plus_enet.predict(cpr11b_c)

Mtk_predictions = pd.DataFrame({'HP1a': a_Mtk_preds,
                                'HP1B': b_Mtk_preds,
                                'HP1C': c_Mtk_preds})
Mtk_predictions.to_csv('Mtk_HP1Plus_sim_predictions.csv', index = False)